<a href="https://colab.research.google.com/github/kleczekr/tolkenizer/blob/master/fantastic_quirks_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
from tabulate import tabulate
from collections import Counter
import pandas as pd

In [2]:
# import nltk, vader, and set up vader sentiment analyzer

import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
# This cell is meant to accommodate the Google Colab way of dealing with reading 
# files from Google Drive; feel free to ignore it if you are running the notebook
# on your local machine
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
counter_nums = ['ONE','TWO','THREE','FOUR','FIVE','SIX','SEVEN','EIGHT','NINE',\
                'TEN','ELEVEN','TWELVE','THIRTEEN','FOURTEEN','FIFTEEN','SIXTEEN',\
                'SEVENTEEN','EIGHTEEN','NINETEEN','TWENTY','TWENTY-ONE','TWENTY-TWO',\
                'TWENTY-THREE','TWENTY-FOUR','TWENTY-FIVE','TWENTY-SIX','TWENTY-SEVEN',\
                'TWENTY-EIGHT','TWENTY-NINE','THIRTY','THIRTY-ONE','THIRTY-TWO', 'THIRTY-THREE',\
                'THIRTY-FOUR']
 
with  open('/content/drive/My Drive/book_txt/killing_floor.txt', 'r') as f:
  kf = f.read()

In [6]:
kf_ch = list()
for i in range(33):
  kf_ch.append(kf.split(counter_nums[i])[1].split(counter_nums[i+1])[0])
 
kf_ch.append(kf.split('THIRTY-FOUR')[1])

In [8]:
# retrieve a list of paragraphs for each chapter!
kf_paras = list()
# create empty lists for each chapter---later you will populate em with paragraphs
for i in range(34):
  kf_paras.append([])

for i in range(34):
  for element in kf_ch[i].splitlines():
    if len(element) > 50:
      kf_paras[i].append(element)
      print(f'added paragraph of length {len(element)}')
    else:
      print(f'rejected paragraph of length {len(element)}')
  print(f'\nfinished processing chapter {i+1}\n')

Streaming output truncated to the last 5000 lines.
rejected paragraph of length 0
added paragraph of length 141
rejected paragraph of length 0
added paragraph of length 110
rejected paragraph of length 0
added paragraph of length 94
rejected paragraph of length 0
added paragraph of length 55
rejected paragraph of length 0
added paragraph of length 100
rejected paragraph of length 0
added paragraph of length 227
rejected paragraph of length 0
added paragraph of length 480
rejected paragraph of length 0
added paragraph of length 152
rejected paragraph of length 0
added paragraph of length 109
rejected paragraph of length 0
added paragraph of length 52
rejected paragraph of length 0
added paragraph of length 127
rejected paragraph of length 0
added paragraph of length 147
rejected paragraph of length 0
added paragraph of length 157
rejected paragraph of length 0
added paragraph of length 256
rejected paragraph of length 0
added paragraph of length 118
rejected paragraph of length 0
added 

In [10]:
kf_spc = list()

# create empty lists for each chapter---later you will populate em with paragraphs
for i in range(34):
  kf_spc.append([])

# now retrieve spacy objects
for i in range(34):
  for another_i in range(len(kf_paras[i])):
    kf_spc[i].append(nlp(kf_paras[i][another_i]))
    print(f'finished processing paragraph {another_i+1} in chapter {i+1}')
  print(f'\nfinished processing chapter {i+1}\n')

finished processing paragraph 1 in chapter 1
finished processing paragraph 2 in chapter 1
finished processing paragraph 3 in chapter 1
finished processing paragraph 4 in chapter 1
finished processing paragraph 5 in chapter 1
finished processing paragraph 6 in chapter 1
finished processing paragraph 7 in chapter 1
finished processing paragraph 8 in chapter 1
finished processing paragraph 9 in chapter 1
finished processing paragraph 10 in chapter 1
finished processing paragraph 11 in chapter 1
finished processing paragraph 12 in chapter 1
finished processing paragraph 13 in chapter 1
finished processing paragraph 14 in chapter 1
finished processing paragraph 15 in chapter 1
finished processing paragraph 16 in chapter 1
finished processing paragraph 17 in chapter 1
finished processing paragraph 18 in chapter 1
finished processing paragraph 19 in chapter 1
finished processing paragraph 20 in chapter 1
finished processing paragraph 21 in chapter 1
finished processing paragraph 22 in chapter

In [18]:
list_pos = list()
chapter_count = 1
# retrieve POS counts, sentence counts and token counts for all the chapters
for chapter in kf_spc:
  for paragraph in chapter:
    poscount = Counter()
    poscount['chapter'] = chapter_count
    poscount['text'] = paragraph.text
    poscount['sentiment'] = sid.polarity_scores(paragraph.text)['compound']
    for sent in paragraph.sents:
      poscount['sentence_count'] += 1
      for tok in sent:
        poscount[tok.pos_] += 1
        poscount['token_count'] += 1
  # you store the Counter in a list of counters
    list_pos.append(poscount)
  chapter_count += 1

In [19]:
# It's difficult to work with list of counters. Convert it to DataFrame
# to perform easy and quick analyses. Remember to fill NaN values
pos_df = pd.DataFrame(list_pos).fillna(0)
# change name of the index
pos_df.index.names = ['paragraph']

In [21]:
# Perform some calculations---raw numbers don't give you much
# pos_df['adj_per_sent'] = pos_df.ADJ/pos_df.sentence_count
pos_df['adj_proportion'] = pos_df.ADJ/pos_df.token_count
# pos_df['verb_per_sent'] = pos_df.VERB/pos_df.sentence_count
pos_df['verb_proportion'] = pos_df.VERB/pos_df.token_count
# pos_df['noun_per_sent'] = pos_df.NOUN/pos_df.sentence_count
pos_df['noun_proportion'] = pos_df.NOUN/pos_df.token_count
# pos_df['adv_per_sent'] = pos_df.ADV/pos_df.sentence_count
pos_df['adv_proportion'] = pos_df.ADV/pos_df.token_count
# pos_df['propn_per_sent'] = pos_df.PROPN/pos_df.sentence_count
pos_df['propn_proportion'] = pos_df.PROPN/pos_df.token_count
pos_df['avg_sentence_len'] = pos_df.token_count/pos_df.sentence_count
# add: cconj, punct, adp, aux, sconj, det
# coordinating conjunction, e.g. and, or, but
pos_df['coord_conj_proportion'] = pos_df.CCONJ/pos_df.token_count
pos_df['punct_proportion'] = pos_df.PUNCT/pos_df.token_count
# adposition---in, to, during
pos_df['adposition_proportion'] = pos_df.ADP/pos_df.token_count
# auxiliary---is, has(done), should (do)
pos_df['auxiliary_proportion'] = pos_df.AUX/pos_df.token_count
# subordinating conjunction---if, while, that
pos_df['sub_conj_proportion'] = pos_df.SCONJ/pos_df.token_count
# determiner---a, an, the
pos_df['det_proportion'] = pos_df.DET/pos_df.token_count

In [22]:
pos_df = pos_df.drop(columns=['PROPN', 'CCONJ', 'PUNCT',	'ADP', 'NOUN', 'NUM', 'AUX', 'ADJ', \
                     'PART', 'VERB', 'SCONJ', 'PRON', 'ADV', 'DET', 'INTJ', 'X', 'SYM'])

In [23]:
# add several more columns to the dataframe---counts of standard deviations above/below
# the mean for: proportion of adjectives (std_adj), proportion of verbs (std_v), proportion of nouns (std_n),
# proportion of adverbs (std_adv) and proportion of proper nouns (std_pn)
pos_df['std_adj'] = (pos_df.adj_proportion - pos_df.adj_proportion.mean(axis=0)) / pos_df.adj_proportion.std(axis=0)
pos_df['std_v'] = (pos_df.verb_proportion - pos_df.verb_proportion.mean(axis=0)) / pos_df.verb_proportion.std(axis=0)
pos_df['std_n'] = (pos_df.noun_proportion - pos_df.noun_proportion.mean(axis=0)) / pos_df.noun_proportion.std(axis=0)
pos_df['std_adv'] = (pos_df.adv_proportion - pos_df.adv_proportion.mean(axis=0)) / pos_df.adv_proportion.std(axis=0)
pos_df['std_pn'] = (pos_df.propn_proportion - pos_df.propn_proportion.mean(axis=0)) / pos_df.propn_proportion.std(axis=0)

# add additional calculations: std_coord, std_punct, std_adp, std_aux, std_sub, std_det
pos_df['std_coord'] = (pos_df.coord_conj_proportion - pos_df.coord_conj_proportion.mean(axis=0)) / pos_df.coord_conj_proportion.std(axis=0)
pos_df['std_punct'] = (pos_df.punct_proportion - pos_df.punct_proportion.mean(axis=0)) / pos_df.punct_proportion.std(axis=0)
pos_df['std_adp'] = (pos_df.adposition_proportion - pos_df.adposition_proportion.mean(axis=0)) / pos_df.adposition_proportion.std(axis=0)
pos_df['std_aux'] = (pos_df.auxiliary_proportion - pos_df.auxiliary_proportion.mean(axis=0)) / pos_df.auxiliary_proportion.std(axis=0)
pos_df['std_sub'] = (pos_df.sub_conj_proportion - pos_df.sub_conj_proportion.mean(axis=0)) / pos_df.sub_conj_proportion.std(axis=0)
pos_df['std_det'] = (pos_df.det_proportion - pos_df.det_proportion.mean(axis=0)) / pos_df.det_proportion.std(axis=0)

In [24]:
# calculate weirdness of a sentence---the sum of absolute values of the calculated number
# of standard deviations from the mean that you've calculated before.
pos_df['original_quirk'] = abs(pos_df.std_adj) + abs(pos_df.std_v) + \
abs(pos_df.std_n) + abs(pos_df.std_adv) + abs(pos_df.std_pn)

# updated quirk metric---more variables taken into consideration
pos_df['updated_quirk'] = abs(pos_df.std_adj) + abs(pos_df.std_v) + abs(pos_df.std_n) + \
abs(pos_df.std_adv) + abs(pos_df.std_pn) + abs(pos_df.std_coord) + abs(pos_df.std_punct)/2 + \
abs(pos_df.std_adp) + abs(pos_df.std_aux)*2 + abs(pos_df.std_sub) + abs(pos_df.std_det)/2

# even more updated quirk metric
pos_df['non_abs_quirk'] = pos_df.std_adj + pos_df.std_v + pos_df.std_n + \
pos_df.std_adv + pos_df.std_pn + pos_df.std_coord + pos_df.std_punct/2 + \
pos_df.std_adp + pos_df.std_aux*2 + pos_df.std_sub + pos_df.std_det/2

In [ ]:
# for display purposes---reduce the length of the text to 50 characters
# pos_df.text = pos_df.text.str.slice(0,30)+'...'

In [35]:
print(tabulate(pos_df[(pos_df.original_quirk>8) & (pos_df.token_count>30)], headers='keys'))

  paragraph    chapter  text                                                                                                                                                                                                                                                     sentiment    sentence_count    token_count    adj_proportion    verb_proportion    noun_proportion    adv_proportion    propn_proportion    avg_sentence_len    coord_conj_proportion    punct_proportion    adposition_proportion    auxiliary_proportion    sub_conj_proportion    det_proportion    std_adj      std_v     std_n    std_adv      std_pn    std_coord    std_punct    std_adp    std_aux    std_sub    std_det    original_quirk    updated_quirk    non_abs_quirk
-----------  ---------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
print(tabulate(pos_df[(pos_df.updated_quirk>14) & (pos_df.token_count>30)], headers='keys'))

  paragraph    chapter  text                                                                                                                                                                                                        sentiment    sentence_count    token_count    adj_proportion    verb_proportion    noun_proportion    adv_proportion    propn_proportion    avg_sentence_len    coord_conj_proportion    punct_proportion    adposition_proportion    auxiliary_proportion    sub_conj_proportion    det_proportion    std_adj      std_v     std_n    std_adv     std_pn    std_coord    std_punct    std_adp     std_aux    std_sub    std_det    original_quirk    updated_quirk    non_abs_quirk
-----------  ---------  --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------  ----------------  -------------  ----------------  -------------

In [40]:
print(tabulate(pos_df[(pos_df.non_abs_quirk>6) & (pos_df.token_count>30)], headers='keys'))

  paragraph    chapter  text                                                                                                                                                                                                             sentiment    sentence_count    token_count    adj_proportion    verb_proportion    noun_proportion    adv_proportion    propn_proportion    avg_sentence_len    coord_conj_proportion    punct_proportion    adposition_proportion    auxiliary_proportion    sub_conj_proportion    det_proportion    std_adj      std_v      std_n    std_adv     std_pn    std_coord    std_punct    std_adp    std_aux    std_sub    std_det    original_quirk    updated_quirk    non_abs_quirk
-----------  ---------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------  ----------------  -------------  ----------------  ---

In [42]:
print(tabulate(pos_df[(pos_df.non_abs_quirk<-5) & (pos_df.token_count>30)], headers='keys'))

  paragraph    chapter  text                                                                                                                                                                                                         sentiment    sentence_count    token_count    adj_proportion    verb_proportion    noun_proportion    adv_proportion    propn_proportion    avg_sentence_len    coord_conj_proportion    punct_proportion    adposition_proportion    auxiliary_proportion    sub_conj_proportion    det_proportion    std_adj       std_v      std_n    std_adv      std_pn    std_coord    std_punct    std_adp    std_aux    std_sub    std_det    original_quirk    updated_quirk    non_abs_quirk
-----------  ---------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------  ----------------  -------------  ----------------  ---------